In [1]:
import os
os.chdir("../")
%pwd

'd:\\Final-Year-Project\\Credit-Card-Fraud-Detection-Using-GNN'

In [2]:
# Entity

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [3]:
from Credit_Card_Fraud_Detection.constants import *
from Credit_Card_Fraud_Detection.utils.common import read_yaml, create_directories

In [4]:
# Configuration

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [5]:
import os
import numpy as np
import pandas as pd
from Credit_Card_Fraud_Detection import logger
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [6]:
#Components

class DataTransformation:
    def __init__(self, config):
        self.config = config
        self.label_encoders = {}
        self.scaler = StandardScaler()

    def load_data(self):
        try:
            df = pd.read_csv(self.config.data_path)
            logger.info("Data loaded successfully.")
            return df
        except FileNotFoundError:
            logger.error(f"File not found: {self.config.data_path}")
            return None

    def create_card_ids(self, df):
        """Uses cc_num as card_id."""
        if 'cc_num' in df.columns:
            df.rename(columns={'cc_num': 'card_id'}, inplace=True)
            logger.info("cc_num column renamed to card_id.")
        elif 'card_id' in df.columns:
            logger.info("card_id column already exists.")
        else:
            logger.error("cc_num column not found. Please check your data.")
            return None # Return None to indicate an error

        return df
    
    def drop_unwanted_columns(self, df):
        df = df.drop(columns=["merch_zipcode", "Unnamed: 0", "trans_num", "unix_time"], errors="ignore") # added trans_num and unix_time
        logger.info("Unwanted columns dropped.")
        return df

    def handle_missing_categorical(self, df):
        df["category"] = df["category"].fillna("unknown")
        df["state"] = df["state"].fillna("unknown")
        logger.info("Missing categorical values handled.")
        return df

    def encode_categorical(self, df):
        if "category" not in self.label_encoders:
            self.label_encoders["category"] = LabelEncoder()
            df["category"] = self.label_encoders["category"].fit_transform(df["category"])
        else:
            df["category"] = self.label_encoders["category"].transform(df["category"])

        if "state" not in self.label_encoders:
            self.label_encoders["state"] = LabelEncoder()
            df["state"] = self.label_encoders["state"].fit_transform(df["state"])
        else:
            df["state"] = self.label_encoders["state"].transform(df["state"])

        logger.info("Categorical columns encoded.")
        return df

    def handle_numerical_features(self, df):
        num_features = ["amt", "city_pop", "lat", "long", "merch_lat", "merch_long"] # removed unix_time
        existing_features = [col for col in num_features if col in df.columns]

        df[existing_features] = df[existing_features].apply(pd.to_numeric, errors='coerce').fillna(0)
        logger.info("Numerical features converted and missing values handled.")

        return df

    def normalize_numerical_features(self, df):
        num_features = ["amt", "city_pop", "lat", "long", "merch_lat", "merch_long"]
        existing_features = [col for col in num_features if col in df.columns]

        df[existing_features] = self.scaler.fit_transform(df[existing_features])
        logger.info("Numerical features normalized.")

        # Round lat, long, merch_lat, merch_long to 3 decimal places
        for col in ["lat", "long", "merch_lat", "merch_long"]:
            if col in df.columns:
                df[col] = df[col].round(3)

        # Handle trans_num conversion (hex to int) and rounding
        if 'trans_num' in df.columns:
            try:
                df["trans_num"] = df["trans_num"].apply(lambda x: int(x, 16))
                df["trans_num"] = df["trans_num"].round() # Round to nearest integer
            except ValueError as e:
                logger.error(f"Error converting hex to int: {e}")
                df = df[df["trans_num"].apply(lambda x: isinstance(x, str) and all(c in '0123456789abcdefABCDEF' for c in x))]
                df["trans_num"] = df["trans_num"].apply(lambda x: int(x, 16)).round()
            except OverflowError as e:
                logger.error(f"Overflow error: {e}")
                df = df[df["trans_num"].apply(lambda x: isinstance(x, str))]
                df["trans_num"] = df["trans_num"].apply(lambda x: pd.to_numeric(x, errors='coerce')).round()
                df= df.dropna(subset=['trans_num'])
                df["trans_num"] = df["trans_num"].astype(int)

        return df

    def preprocess(self):
        df = self.load_data()
        if df is None:
            return

        df = self.drop_unwanted_columns(df)
        df = self.handle_missing_categorical(df)
        df = self.encode_categorical(df)
        df = self.handle_numerical_features(df)
        df = self.normalize_numerical_features(df)

        # Save preprocessed dataset
        preprocessed_path = os.path.join(self.config.root_dir, "transformed_dataset.csv")
        df.to_csv(preprocessed_path, index=False)
        logger.info(f"Preprocessed data saved at {preprocessed_path}")

        return df  # return the dataframe for further processing

In [7]:
# Pipeline

try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    transformed_data = data_transformation.preprocess()  # Corrected method call
except Exception as e:
    raise e  # This will re-raise the exception if any error occurs

[2025-03-18 00:53:03,681: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-18 00:53:03,685: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-18 00:53:03,687: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-18 00:53:03,688: INFO: common: created directory at: artifacts]
[2025-03-18 00:53:03,688: INFO: common: created directory at: artifacts/data_transformation]
[2025-03-18 00:53:07,328: INFO: 3722950385: Data loaded successfully.]
[2025-03-18 00:53:07,409: INFO: 3722950385: Unwanted columns dropped.]
[2025-03-18 00:53:07,545: INFO: 3722950385: Missing categorical values handled.]
[2025-03-18 00:53:07,818: INFO: 3722950385: Categorical columns encoded.]
[2025-03-18 00:53:07,884: INFO: 3722950385: Numerical features converted and missing values handled.]
[2025-03-18 00:53:08,022: INFO: 3722950385: Numerical features normalized.]
[2025-03-18 00:53:17,097: INFO: 3722950385: Preprocessed data saved at artifacts/data_transformation